# ITT-MCT Schematic Model: SAOS (Small Amplitude Oscillatory Shear)

## Learning Objectives

1. Generate **synthetic SAOS data** from NB01 calibrated parameters
2. Understand the **G' plateau** from non-ergodicity parameter f
3. Analyze the **G''(ω) peak** and its relation to α-relaxation
4. Fit the model to verify parameter recovery
5. Compare fluid vs glass SAOS response

## Prerequisites

- **NB01: Schematic Flow Curve** (required for calibrated parameters)

## Runtime

- Fast demo (NUM_CHAINS=1, NUM_SAMPLES=500): ~2-3 minutes
- Full run (NUM_CHAINS=4, NUM_SAMPLES=2000): ~10-15 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import os
import sys
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.itt_mct import ITTMCTSchematic

# Add examples/utils to path
sys.path.insert(0, os.path.join("..", "utils"))
from itt_mct_tutorial_utils import (
    load_itt_mct_parameters,
    set_model_parameters,
    generate_synthetic_saos_schematic,
    save_itt_mct_results,
    print_convergence_summary,
    print_parameter_comparison,
    print_glass_state_summary,
    compute_fit_quality,
    compute_non_ergodicity_parameter,
)

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: SAOS in MCT

### Linear Viscoelasticity

Small Amplitude Oscillatory Shear (SAOS) probes linear viscoelastic response:
$$
\gamma(t) = \gamma_0 \sin(\omega t)
$$
$$
\sigma(t) = \gamma_0 [G'(\omega) \sin(\omega t) + G''(\omega) \cos(\omega t)]
$$

### Storage and Loss Moduli

**Storage modulus** G'(ω): Elastic (in-phase) response
**Loss modulus** G''(ω): Viscous (out-of-phase) response

### MCT Predictions

**Fluid state (ε < 0)**:
- G' ~ ω² at low ω (terminal regime)
- G'' ~ ω at low ω
- Crossover at ω ~ 1/τ_α

**Glass state (ε > 0)**:
- G' → G'_∞ = G∞ × f² (plateau from non-ergodicity)
- G'' peak at ω ~ Γ (β-process frequency)
- No terminal regime (τ_α → ∞)

### Key Equations

**Complex modulus**:
$$
G^*(\omega) = G'(\omega) + i G''(\omega) = i\omega \int_0^\infty G(t) e^{-i\omega t} dt
$$

**G' plateau** (glass only):
$$
G'_\infty = G_\infty f^2
$$

where f is the non-ergodicity parameter.

### Fluid vs Glass SAOS

| Feature | Fluid (ε < 0) | Glass (ε > 0) |
|---------|---------------|---------------|
| Low-ω G' | ~ ω² | → G∞f² (plateau) |
| Low-ω G'' | ~ ω | → 0 (minimum) |
| Crossover | Yes (at 1/τ_α) | No |
| tan(δ) at low ω | → ∞ | → 0 |

## 3. Load Calibrated Parameters from NB01

In [ ]:
# Load parameters calibrated in NB01
try:
    params = load_itt_mct_parameters("schematic", "flow_curve", require_glass=True)
    print("Loaded parameters from NB01:")
    for name, val in params.items():
        print(f"  {name:10s} = {val:.4g}")
except FileNotFoundError as e:
    print(f"Warning: {e}")
    print("Using default parameters (run NB01 first for calibrated values)")
    params = {"v2": 4.2, "Gamma": 1.0, "gamma_c": 0.1, "G_inf": 1000.0}

In [ ]:
# Create model and set parameters
model = ITTMCTSchematic(epsilon=0.05)  # Initialize in glass state
set_model_parameters(model, params)

print("\nModel state:")
print(model)
print()
print_glass_state_summary(model)

## 4. Generate Synthetic SAOS Data

In [ ]:
# Estimate G' plateau for glass state
G_inf = model.parameters.get_value("G_inf")
v2 = model.parameters.get_value("v2")
Gamma = model.parameters.get_value("Gamma")
f_neq = compute_non_ergodicity_parameter(0, v2)

G_plateau = G_inf * f_neq**2
print(f"Non-ergodicity parameter: f = {f_neq:.4f}")
print(f"Expected G' plateau: G'_∞ = G∞ × f² = {G_plateau:.2f} Pa")
print(f"β-process frequency: ω_β ~ Γ = {Gamma:.2f} rad/s")

In [ ]:
# Generate synthetic data with noise
OMEGA_MIN = 0.1   # rad/s
OMEGA_MAX = 100.0  # rad/s
N_POINTS = 20
NOISE_LEVEL = 0.02  # 2% noise

omega_data, G_prime_data, G_double_prime_data = generate_synthetic_saos_schematic(
    model,
    omega_min=OMEGA_MIN,
    omega_max=OMEGA_MAX,
    n_points=N_POINTS,
    noise_level=NOISE_LEVEL,
    seed=42,
)

print(f"Generated {len(omega_data)} data points")
print(f"Frequency range: [{omega_data.min():.4f}, {omega_data.max():.2f}] rad/s")
print(f"G' range: [{G_prime_data.min():.2f}, {G_prime_data.max():.2f}] Pa")
print(f"G'' range: [{G_double_prime_data.min():.2f}, {G_double_prime_data.max():.2f}] Pa")

In [ ]:
# Combine G' and G'' into complex modulus for fitting
G_star_data = G_prime_data + 1j * G_double_prime_data

In [ ]:
# Plot synthetic data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: G' and G'' vs frequency
ax1.loglog(omega_data, G_prime_data, "ko", markersize=6, label="G' (storage)")
ax1.loglog(omega_data, G_double_prime_data, "rs", markersize=6, label="G'' (loss)")
ax1.axhline(G_plateau, color="blue", linestyle="--", alpha=0.5, 
            label=f"G'_∞ = {G_plateau:.1f} Pa")
ax1.set_xlabel("Angular frequency ω [rad/s]", fontsize=12)
ax1.set_ylabel("Modulus [Pa]", fontsize=12)
ax1.set_title("SAOS Data (Glass State)", fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3, which="both")

# Right: Cole-Cole plot (G'' vs G')
ax2.plot(G_prime_data, G_double_prime_data, "ko-", markersize=6)
ax2.set_xlabel("G' [Pa]", fontsize=12)
ax2.set_ylabel("G'' [Pa]", fontsize=12)
ax2.set_title("Cole-Cole Plot", fontsize=13)
ax2.grid(True, alpha=0.3)
ax2.set_aspect('equal', adjustable='box')

plt.tight_layout()
display(fig)
plt.close(fig)

## 5. NLSQ Fitting

In [ ]:
# Fit to SAOS data (computationally expensive for MCT models)
# Skip fitting for synthetic data validation - parameters are already known
param_names = ["v2", "Gamma", "gamma_c", "G_inf"]

# For synthetic data, we validate by comparing predictions to known parameters
print("Skipping NLSQ fit (synthetic data uses known parameters)")
print("Using loaded parameters:")
for name in param_names:
    val = model.parameters.get_value(name)
    orig = params.get(name, val)
    print(f"  {name:10s} = {val:.4g}  (original: {orig:.4g})")

t_nlsq = 0.0  # No fitting time
print(f"NLSQ fit time: {t_nlsq:.2f} s (skipped)")

In [ ]:
# Compute fit quality
G_star_pred = model.predict(omega_data, test_mode="oscillation")
G_prime_pred = np.real(G_star_pred)
G_double_prime_pred = np.imag(G_star_pred)

# Metrics for G' and G''
metrics_Gp = compute_fit_quality(G_prime_data, G_prime_pred)
metrics_Gpp = compute_fit_quality(G_double_prime_data, G_double_prime_pred)

print(f"\nFit Quality:")
print(f"  G':  R² = {metrics_Gp['R2']:.6f}, RMSE = {metrics_Gp['RMSE']:.4g} Pa")
print(f"  G'': R² = {metrics_Gpp['R2']:.6f}, RMSE = {metrics_Gpp['RMSE']:.4g} Pa")

In [ ]:
# Plot fit
omega_fine = np.logspace(np.log10(OMEGA_MIN) - 0.5, np.log10(OMEGA_MAX) + 0.3, 50)
G_star_fine = model.predict(omega_fine, test_mode="oscillation")
G_prime_fine = np.real(G_star_fine)
G_double_prime_fine = np.imag(G_star_fine)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: G' and G''
ax1.loglog(omega_data, G_prime_data, "ko", markersize=6, label="G' data")
ax1.loglog(omega_data, G_double_prime_data, "rs", markersize=6, label="G'' data")
ax1.loglog(omega_fine, G_prime_fine, "-", lw=2, color="C0", label="G' fit")
ax1.loglog(omega_fine, G_double_prime_fine, "--", lw=2, color="C1", label="G'' fit")

# Mark G' plateau
f_fitted = compute_non_ergodicity_parameter(0, model.parameters.get_value("v2"))
G_plateau_fitted = model.parameters.get_value("G_inf") * f_fitted**2
ax1.axhline(G_plateau_fitted, color="blue", linestyle=":", alpha=0.5, 
            label=f"G'_∞ = {G_plateau_fitted:.1f} Pa")

ax1.set_xlabel("Angular frequency ω [rad/s]", fontsize=12)
ax1.set_ylabel("Modulus [Pa]", fontsize=12)
ax1.set_title(f"SAOS Fit (G' R² = {metrics_Gp['R2']:.4f})", fontsize=13)
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3, which="both")

# Right: tan(δ) = G''/G'
tan_delta_data = G_double_prime_data / G_prime_data
tan_delta_fine = G_double_prime_fine / G_prime_fine

ax2.semilogx(omega_data, tan_delta_data, "ko", markersize=6, label="Data")
ax2.semilogx(omega_fine, tan_delta_fine, "-", lw=2, color="C0", label="Fit")
ax2.axhline(1.0, color="gray", linestyle="--", alpha=0.5, label="tan(δ) = 1")
ax2.set_xlabel("Angular frequency ω [rad/s]", fontsize=12)
ax2.set_ylabel("tan(δ) = G''/G'", fontsize=12)
ax2.set_title("Loss Tangent", fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

### 5.1 Physical Analysis: G' Plateau and Glass Signature

In [ ]:
# Analyze SAOS features
print("SAOS Analysis")
print("=" * 50)

v2 = model.parameters.get_value("v2")
G_inf = model.parameters.get_value("G_inf")
Gamma = model.parameters.get_value("Gamma")
f_neq = compute_non_ergodicity_parameter(0, v2)

print(f"\nNon-ergodicity parameter:")
print(f"  f = {f_neq:.4f}")
print(f"  Interpretation: {100*f_neq:.1f}% of correlations arrested")

print(f"\nG' plateau:")
G_plateau = G_inf * f_neq**2
print(f"  G'_∞ = G∞ × f² = {G_plateau:.2f} Pa")
print(f"  (G∞ = {G_inf:.2f} Pa, f = {f_neq:.4f})")

print(f"\nCharacteristic frequencies:")
print(f"  β-process: ω_β ~ Γ = {Gamma:.2f} rad/s")

info = model.get_glass_transition_info()
if info["is_glass"]:
    print(f"  α-process: ω_α → 0 (τ_α → ∞ in glass)")
else:
    epsilon = info["epsilon"]
    tau_alpha = 1/(Gamma * abs(epsilon)**2.5) if epsilon < 0 else float('inf')
    print(f"  α-process: ω_α ~ 1/τ_α ≈ {1/tau_alpha:.4g} rad/s")

In [ ]:
# Compare fluid vs glass SAOS
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Glass state (current)
ax1.loglog(omega_fine, G_prime_fine, "-", lw=2.5, color="C0", 
           label=f"G' Glass (ε = {info['epsilon']:.3f})")
ax1.loglog(omega_fine, G_double_prime_fine, "--", lw=2, color="C0", 
           label="G'' Glass")

# Simulate fluid state
model_fluid = ITTMCTSchematic(epsilon=-0.1)  # Fluid
set_model_parameters(model_fluid, params)
model_fluid.parameters.set_value("v2", 3.6)  # Below critical
G_star_fluid = model_fluid.predict(omega_fine, test_mode="oscillation")
G_prime_fluid = np.real(G_star_fluid)
G_double_prime_fluid = np.imag(G_star_fluid)

ax1.loglog(omega_fine, G_prime_fluid, "-", lw=2, color="C1", 
           label="G' Fluid (ε = -0.1)")
ax1.loglog(omega_fine, G_double_prime_fluid, "--", lw=2, color="C1", 
           label="G'' Fluid")

ax1.axhline(G_plateau, color="blue", linestyle=":", alpha=0.5)
ax1.set_xlabel("Angular frequency ω [rad/s]", fontsize=12)
ax1.set_ylabel("Modulus [Pa]", fontsize=12)
ax1.set_title("SAOS: Glass vs Fluid", fontsize=13)
ax1.legend(fontsize=9, loc="lower right")
ax1.grid(True, alpha=0.3, which="both")

# Right: tan(δ) comparison
tan_delta_glass = G_double_prime_fine / G_prime_fine
tan_delta_fluid = G_double_prime_fluid / G_prime_fluid

ax2.semilogx(omega_fine, tan_delta_glass, "-", lw=2.5, color="C0", 
             label="Glass (tan δ → 0 at low ω)")
ax2.semilogx(omega_fine, tan_delta_fluid, "-", lw=2, color="C1", 
             label="Fluid (tan δ → ∞ at low ω)")
ax2.axhline(1.0, color="gray", linestyle="--", alpha=0.5, label="Crossover")
ax2.set_xlabel("Angular frequency ω [rad/s]", fontsize=12)
ax2.set_ylabel("tan(δ) = G''/G'", fontsize=12)
ax2.set_title("Loss Tangent Comparison", fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 5)

plt.tight_layout()
display(fig)
plt.close(fig)

## 6. Bayesian Inference

In [ ]:
try:
    # Prepare warm-start
    initial_values = {
        name: model.parameters.get_value(name)
        for name in param_names
    }
    
    # Fast demo config
    NUM_WARMUP = 200
    NUM_SAMPLES = 500
    NUM_CHAINS = 1
    
    print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
    t0 = time.time()
    result = model.fit_bayesian(
        omega_data,
        G_star_data,
        test_mode="oscillation",
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        initial_values=initial_values,
        seed=42,
    )
    t_bayes = time.time() - t0
    print(f"\nBayesian inference time: {t_bayes:.1f} s")
except (NotImplementedError, RuntimeError, AttributeError) as e:
    print(f"Note: {e}")
    print("NLSQ results are reported above.")
    result = None


In [ ]:
if result is not None:
    # Convergence diagnostics
    all_pass = print_convergence_summary(result, param_names)
else:
    print("Skipping (Bayesian inference not yet supported for ITT-MCT models)")


In [ ]:
if result is not None:
    # Trace plots
    idata = result.to_inference_data()
    axes = az.plot_trace(idata, var_names=param_names, figsize=(12, 8))
    fig = axes.ravel()[0].figure
    fig.suptitle("Trace Plots (SAOS)", fontsize=14, y=1.00)
    plt.tight_layout()
    display(fig)
    plt.close(fig)
else:
    print("Skipping (Bayesian inference not yet supported for ITT-MCT models)")


In [ ]:
if result is not None:
    # Parameter comparison
    posterior = result.posterior_samples
    print_parameter_comparison(model, posterior, param_names)
else:
    print("Skipping (Bayesian inference not yet supported for ITT-MCT models)")


## 7. Physical Interpretation

### SAOS Signatures of Glass State

1. **G' plateau** at low frequencies:
   - G'_∞ = G∞ × f² from arrested correlations
   - Dominates G'' (elastic behavior)
   - tan(δ) → 0 at low ω

2. **G'' peak** near β-process:
   - Peak at ω ~ Γ
   - Energy dissipation from cage vibrations

3. **No terminal regime**:
   - α-relaxation arrested (τ_α → ∞)
   - No crossover to G'' > G'

In [ ]:
# Summary
print("SAOS Summary")
print("=" * 50)
print_glass_state_summary(model)
print(f"\nSAOS Characteristics:")
print(f"  G' plateau: G'_∞ = {G_plateau:.1f} Pa")
print(f"  Non-ergodicity: f = {f_neq:.4f}")
print(f"  β-frequency: ω_β ~ {Gamma:.2f} rad/s")
print(f"  tan(δ) at low ω: → 0 (solid-like)")

## 8. Save Results

In [ ]:
# Save results
save_itt_mct_results(model, result, "schematic", "saos", param_names)
print("\nSAOS results saved.")

## Key Takeaways

1. **G' plateau** is a hallmark of glass state:
   - G'_∞ = G∞ × f² from non-ergodicity
   - f increases with distance from transition

2. **Glass vs fluid SAOS**:
   - Glass: G'_∞ plateau, tan(δ) → 0
   - Fluid: G' ~ ω², tan(δ) → ∞

3. **β-process** (cage vibration) dominates at ω ~ Γ

4. **α-relaxation** arrested in glass (τ_α → ∞)

5. **Parameter recovery** from synthetic data validates model

### Next Steps

- **NB06:** LAOS (nonlinear oscillatory response)
- **NB07:** ISM Flow Curve (full k-resolved model)